In [1]:
# libraries for importing and editing Twitter tweets
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import re

# dataframe nad numbers libraries
import pandas as pd
import numpy as np

# visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns

# crypto-coins api
import cryptocompare

In [2]:
api_log = pd.read_csv('twitter_api_login.csv')

In [3]:
# Twitter API credentials
api_key = api_log.iloc[0, 0]
api_key_secret = api_log.iloc[0, 1]
bearer_token = api_log.iloc[0, 2]
access_token = api_log.iloc[0, 3]
access_token_secret = api_log.iloc[0, 4]

In [4]:
# Create authentication object
authenticate = tweepy.OAuthHandler(api_key, api_key_secret)

# Set access token and access token secret
authenticate.set_access_token(access_token, access_token_secret)

# create the API object while passing in the auth information
api = tweepy.API(authenticate, wait_on_rate_limit=True)

In [5]:
cryptocompare.cryptocompare._set_api_key_parameter('c22260e3802579ea1080ec1830c1b4ec07f2c449b47b21ab632e44664c102746')

'&api_key=c22260e3802579ea1080ec1830c1b4ec07f2c449b47b21ab632e44664c102746'

In [6]:
coin_list = cryptocompare.get_coin_list(format=False)
convert_to_list = [list(coin) for coin in list(coin_list.items())]
# coin_list = [a_coin[1]['CoinName'] for a_coin in convert_to_list]
coin_list_top = []
print(len(convert_to_list))
for coin in convert_to_list:
    if int(coin[1]['SortOrder']) >= 0:
        coin_list_top.append((int(coin[1]['SortOrder']), coin[1]['CoinName']))
coin_list_top.sort()
coin_list_top

8494


[(7872, 'CateCoin'),
 (7873, 'Catgirl'),
 (7874, 'DogeZilla'),
 (7875, 'Taboo Token'),
 (7876, 'Aurory'),
 (7877, 'Gods Unchained'),
 (7878, 'Green Shiba Inu'),
 (7879, 'Bitspawn'),
 (7880, 'Pist Trust'),
 (7881, 'Pando'),
 (7882, 'Mines of Dalarnia'),
 (7883, 'Space SIP'),
 (7884, 'Mononoke Inu'),
 (7885, 'Virtue Poker Points'),
 (7886, 'Privapp Network'),
 (7887, 'Pocoland'),
 (7888, 'Super Floki'),
 (7889, 'ENVOY'),
 (7890, 'Planet Finance'),
 (7891, 'NEET Finance'),
 (7892, 'Tranchess'),
 (7893, 'Immutable X'),
 (7894, 'Bifrost Native Coin'),
 (7895, 'Merit Circle'),
 (7896, 'Mist'),
 (7897, 'HurricaneSwap Token'),
 (7898, 'Nobility'),
 (7899, 'YOI SHIBA INU'),
 (7900, 'NFTb'),
 (7901, 'Wifedoge'),
 (7902, 'Idle Mystic'),
 (7903, 'Round Dollar'),
 (7904, 'BTRIPS'),
 (7905, 'Animal Token'),
 (7906, 'Identity'),
 (7907, 'Shibance Token'),
 (7908, 'Ridotto'),
 (7909, 'Nasa Doge'),
 (7910, 'Fastswap'),
 (7911, 'AetherV2'),
 (7912, 'Tiki Token'),
 (7913, 'Parabolic'),
 (7914, 'LaunchZon

In [7]:
import json
from bs4 import BeautifulSoup
import requests

In [8]:
url = 'https://min-api.cryptocompare.com/data/top/totalvolfull?limit=10&tsym=USD'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
data = json.loads(soup.prettify())
data = data['Data']

coin_list = [coin['CoinInfo']['FullName'] for coin in data]
coin_list

['Bitcoin',
 'Ethereum',
 'Solana',
 'Tether',
 'BUSD',
 'XRP',
 'Binance Coin',
 'Cardano',
 'USD Coin',
 'TRON']

In [9]:
# Create function to clean tweets
# must create this clean tweet function later based on my tweets
def clean_twt(twt):
    """"""
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]
    if type(twt) == np.float:
        return ""
    temp = twt.lower()
    temp = re.sub("'", "", temp)  # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+", "", temp)
    temp = re.sub("#[A-Za-z0-9_]+", "", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]', ' ', temp)
    temp = re.sub("[^a-z0-9]", " ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp

In [10]:
# function to get subjectivity
def get_subjectivity(twt):
    blob = TextBlob(twt)
    return blob.sentiment.subjectivity

# function to get polarity
def get_polarity(twt):
    blob = TextBlob(twt)
    return blob.sentiment.polarity

# create function to get sentiment of the tweet text
def get_sentiment(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [12]:
import os
import glob

files = glob.glob('crypto_live_data/*')
for f in files:
    os.remove(f)

In [13]:
coin_list_append = []
no_of_tweets = 100
# enumerate(coin_list) or coin_list_top
for i, coin in enumerate(coin_list):
    coin_search = coin.replace(' ', '').lower()
    search_term = f'#{coin_search} -filter:retweets'
    # Create a cursor object
    tweets = tweepy.Cursor(api.search_tweets, q=search_term, lang='en', tweet_mode='extended').items(no_of_tweets)
    all_tweets = []
    # all_tweets_date = []
    for tweet in tweets:
        # print(tweet.full_text,'\n|| ', tweet.created_at, '\n-----------------------------------')
        all_tweets.append(tweet.full_text)
        # all_tweets_date.append(tweet.created_at)
    extracted_tweets_no = len(all_tweets)
    if extracted_tweets_no < 0:
        continue
    # Create dataframe to store the tweets in a column 'Tweets'
    df_tweets = pd.DataFrame(all_tweets, columns=['Tweets'])
    # create column with cleaned tweets
    df_tweets['Cl_Tweets'] = df_tweets['Tweets'].apply(clean_twt)
    # create two new columns for 'Subjectivity' and 'Polarity'
    df_tweets['Subjectivity'] = df_tweets['Cl_Tweets'].apply(get_subjectivity)
    df_tweets['Polarity'] = df_tweets['Cl_Tweets'].apply(get_polarity)
    # create column to store the text sentiment
    df_tweets['Sentiment'] = df_tweets['Polarity'].apply(get_sentiment)
    df_tweets.to_csv(f"crypto_live_data/{i+1}. {coin}.csv", index=False)
    coin_calcs = [coin, extracted_tweets_no, df_tweets['Subjectivity'].mean(), df_tweets['Polarity'].mean(),
                  df_tweets['Sentiment'].mode()[0]]
    coin_list_append.append(coin_calcs)
    print(i, '.', extracted_tweets_no)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


0 . 100
1 . 100
2 . 100
3 . 100
4 . 8
5 . 100
6 . 100
7 . 100
8 . 100
9 . 100


In [14]:
df = pd.DataFrame(coin_list_append, columns=['Crypto-Coin', 'No. of Tweets', 'Avg Subjectivity', 'Avg Polarity',
                                             'Common Sentiment'])
df

,Crypto-Coin,No. of Tweets,Avg Subjectivity,Avg Polarity,Common Sentiment
0,Bitcoin,100,0.319306,0.073376,Neutral
1,Ethereum,100,0.335525,0.182698,Neutral
2,Solana,100,0.321333,0.129125,Neutral
3,Tether,100,0.265880,0.074296,Neutral
4,BUSD,8,0.442250,0.263015,Positive
5,XRP,100,0.380405,0.142731,Positive
6,Binance Coin,100,0.281255,0.148802,Positive
7,Cardano,100,0.619128,-0.004040,Negative
8,USD Coin,100,0.190172,0.162290,Neutral
9,TRON,100,0.255867,0.055386,Neutral


In [15]:
to_normalize = ['Avg Subjectivity', 'Avg Polarity']

for col in to_normalize:
    df[f'Nor. {col}'] = ((df[col] - df[col].min()) / (df[col].max() - df[col].min())).round(2)
df


,Crypto-Coin,No. of Tweets,Avg Subjectivity,Avg Polarity,Common Sentiment,Nor. Avg Subjectivity,Nor. Avg Polarity
0,Bitcoin,100,0.319306,0.073376,Neutral,0.30,0.29
1,Ethereum,100,0.335525,0.182698,Neutral,0.34,0.70
2,Solana,100,0.321333,0.129125,Neutral,0.31,0.50
3,Tether,100,0.265880,0.074296,Neutral,0.18,0.29
4,BUSD,8,0.442250,0.263015,Positive,0.59,1.00
5,XRP,100,0.380405,0.142731,Positive,0.44,0.55
6,Binance Coin,100,0.281255,0.148802,Positive,0.21,0.57
7,Cardano,100,0.619128,-0.004040,Negative,1.00,0.00
8,USD Coin,100,0.190172,0.162290,Neutral,0.00,0.62
9,TRON,100,0.255867,0.055386,Neutral,0.15,0.22


In [16]:
df_sort = df.sort_values('Nor. Avg Polarity', ascending=False, ignore_index=True)
df_sort

,Crypto-Coin,No. of Tweets,Avg Subjectivity,Avg Polarity,Common Sentiment,Nor. Avg Subjectivity,Nor. Avg Polarity
0,BUSD,8,0.442250,0.263015,Positive,0.59,1.00
1,Ethereum,100,0.335525,0.182698,Neutral,0.34,0.70
2,USD Coin,100,0.190172,0.162290,Neutral,0.00,0.62
3,Binance Coin,100,0.281255,0.148802,Positive,0.21,0.57
4,XRP,100,0.380405,0.142731,Positive,0.44,0.55
5,Solana,100,0.321333,0.129125,Neutral,0.31,0.50
6,Bitcoin,100,0.319306,0.073376,Neutral,0.30,0.29
7,Tether,100,0.265880,0.074296,Neutral,0.18,0.29
8,TRON,100,0.255867,0.055386,Neutral,0.15,0.22
9,Cardano,100,0.619128,-0.004040,Negative,1.00,0.00


In [17]:
df_sort.shape

(10, 7)

In [18]:
df_sort['Crypto-Coin'][0]

'BUSD'

In [19]:
print("Cryptocurrencies arranged in terms of Positivity: \n")
for i in range(df_sort.shape[0]):
    print(f"{i+1}. {df_sort['Crypto-Coin'][i]}")
    print(f"\tMost Common Sentiment:    {df_sort['Common Sentiment'][i]}")
    print(f"\tAverage Positivity:       {round(df_sort['Avg Polarity'][i]*100,1)}%")
    print(f"\tAverage Subjectivity:     {round(df_sort['Avg Subjectivity'][i]*100,1)}%\n")

Cryptocurrencies arranged in terms of Positivity: 

1. BUSD
	Most Common Sentiment:    Positive
	Average Positivity:       26.3%
	Average Subjectivity:     44.2%

2. Ethereum
	Most Common Sentiment:    Neutral
	Average Positivity:       18.3%
	Average Subjectivity:     33.6%

3. USD Coin
	Most Common Sentiment:    Neutral
	Average Positivity:       16.2%
	Average Subjectivity:     19.0%

4. Binance Coin
	Most Common Sentiment:    Positive
	Average Positivity:       14.9%
	Average Subjectivity:     28.1%

5. XRP
	Most Common Sentiment:    Positive
	Average Positivity:       14.3%
	Average Subjectivity:     38.0%

6. Solana
	Most Common Sentiment:    Neutral
	Average Positivity:       12.9%
	Average Subjectivity:     32.1%

7. Bitcoin
	Most Common Sentiment:    Neutral
	Average Positivity:       7.3%
	Average Subjectivity:     31.9%

8. Tether
	Most Common Sentiment:    Neutral
	Average Positivity:       7.4%
	Average Subjectivity:     26.6%

9. TRON
	Most Common Sentiment:    Neutral
	A